# TO DO:
#### 1-Recommender system yapmak
<hr>

In [243]:
import numpy as np
import pandas as pd

In [244]:
df = pd.read_csv('bookdata.csv')

In [245]:
df.drop('Unnamed: 0',axis=1,inplace=True)

# DataFrame Information

In [246]:
df.head(3)

,id,Book Names,Kinds,Authors,Is Fiction,Pages,Ratings
0,3,I Feel Bad About My Neck,horror,Nora Ephron,1,1410,7.9
1,3,Broken Glass,non-fiction novel,Alain Mabanckou,0,332,3.7
2,3,The Girl With the Dragon Tattoo,non-fiction novel,Stieg Larsson,0,177,3.9


In [247]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          6493 non-null   int64  
 1   Book Names  6493 non-null   object 
 2   Kinds       6493 non-null   object 
 3   Authors     6493 non-null   object 
 4   Is Fiction  6493 non-null   int64  
 5   Pages       6493 non-null   int64  
 6   Ratings     6493 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 355.2+ KB


In [248]:
df.describe()

,id,Is Fiction,Pages,Ratings
count,6493.000000,6493.000000,6493.000000,6493.000000
mean,52.976282,0.656245,822.813799,5.021300
std,29.259084,0.474997,408.842476,2.912512
min,3.000000,0.000000,8.000000,0.000000
25%,27.000000,0.000000,567.000000,2.500000
50%,52.000000,1.000000,878.000000,5.000000
75%,79.000000,1.000000,1183.000000,7.500000
max,103.000000,1.000000,1494.000000,10.000000


## RECOMMENDER SYSTEM

## Creating a user-based df

In [249]:
rec_df = df.pivot_table(index='id',values='Ratings',columns='Book Names')

In [250]:
rec_df.head(3)

Book Names,A Death in the Family,A Little Life,A Visit from The Goon Squad,Adults in the Room,Atonement,Austerlitz,Autumn,Bad Blood,Between the World and Me,Broken Glass,...,The Year of Magical Thinking,"Thinking, Fast and Slow",This House of Grief,True History of the Kelly Gang,Underland,Visitation,White Teeth,Why Be Happy When You Could Be Nor,Wolf Hall,Women & Power
id,,,,,,,,,,,,,,,,,,,,,
3,NaN,4.2,NaN,9.6,7.6,NaN,NaN,6.0,NaN,3.7,...,3.8,7.3,3.3,6.4,9.9,1.7,3.9,7.3,NaN,7.4
4,8.9,8.5,0.5,1.5,6.3,NaN,NaN,1.7,NaN,3.2,...,9.3,3.3,4.0,7.0,3.1,4.8,7.7,4.8,NaN,5.4
5,NaN,4.7,NaN,1.8,8.6,NaN,NaN,4.7,NaN,0.2,...,2.6,8.6,5.7,2.0,3.3,1.1,5.4,5.6,NaN,1.1


In [292]:
ratings = pd.DataFrame(df.groupby('Book Names')['Ratings'].mean())

In [293]:
ratings['num of ratings'] = df.groupby('Book Names')['Ratings'].count()

In [294]:
ratings.reset_index(inplace=True)

In [254]:
a_little_life = rec_df['A Little Life']

In [255]:
rec_df.corrwith(rec_df[['A Little Life','Never Let Me Go']]).sort_values(ascending=False)

Book Names
Never Let Me Go                       1.0
A Little Life                         1.0
A Death in the Family                 NaN
A Visit from The Goon Squad           NaN
Adults in the Room                    NaN
                                     ... 
Visitation                            NaN
White Teeth                           NaN
Why Be Happy When You Could Be Nor    NaN
Wolf Hall                             NaN
Women & Power                         NaN
Length: 100, dtype: float64

In [256]:
def get_recs(name):
    try: return dict(rec_df.corrwith(rec_df[name]).sort_values(ascending=False)[1:5])
    except: return False
    

In [257]:
def calc_recs(namelist):
    userdict = {}
    for name in namelist:
        namedict = get_recs(name)
        if not namedict: continue
        for key in namedict.keys():
            if key in namelist: continue
            if key not in userdict.keys():
                userdict[key] = [namedict[key]*10]
            else: 
                userdict[key].append(namedict[key]*10)
    for key in userdict.keys():
        userdict[key] = sum(userdict[key])/len(userdict[key])*10
    return userdict

In [258]:
print('-----------------------------------------------\n| TOP 5 Recommendations based on your readings |\n|                    ----                      |') 
for i,name in enumerate(pd.Series(calc_recs(['Wolf Hall','The Green Road','Adults in the Room','Oryx and Crake'])).sort_values(ascending=False)[:5].keys()):
    print(f'|{i+1}| {name}{(43-len(name))*" "}|\n-----------------------------------------------')
    

-----------------------------------------------
| TOP 5 Recommendations based on your readings |
|                    ----                      |
|1| I Feel Bad About My Neck                   |
-----------------------------------------------
|2| Austerlitz                                 |
-----------------------------------------------
|3| The Shock Doctrine                         |
-----------------------------------------------
|4| Gilead                                     |
-----------------------------------------------
|5| Autumn                                     |
-----------------------------------------------


In [259]:
rec_df.agg('mean').sort_values(ascending=False)[1:5]

Book Names
Outline by Rachel              5.961818
Fun Home                       5.928846
The Emperor of All Maladies    5.703846
Brooklyn                       5.656944
dtype: float64

In [299]:
ratings.head(3)

,Book Names,Ratings,num of ratings
0,A Death in the Family,5.536364,44
1,A Little Life,5.252475,101
2,A Visit from The Goon Squad,4.959375,32


32

In [312]:
def all_calc(rec_df,lencol):
    for col in rec_df.columns:
        if ratings[ratings['Book Names']=='A Visit from The Goon Squad']['num of ratings'].iloc[0]< lencol:
            rec_df = rec_df.drop(col,axis=1)
    return dict(rec_df.agg('mean').sort_values(ascending=False)[1:8])

In [321]:
print()
for i,name in enumerate((all_calc(rec_df,len(rec_df)/48).keys())):
    print(f'|{i + 1}| {name}{(43 - len(name)) * " "}|\n-----------------------------------------------')


|1| Outline by Rachel                          |
-----------------------------------------------
|2| Fun Home                                   |
-----------------------------------------------
|3| The Emperor of All Maladies                |
-----------------------------------------------
|4| Brooklyn                                   |
-----------------------------------------------
|5| The Sixth Extinction                       |
-----------------------------------------------
|6| Chronicles: Volume One                     |
-----------------------------------------------
|7| Experience                                 |
-----------------------------------------------


In [281]:
rec_df['Outline by Rachel'].isna().sum()

46

In [322]:
' '*23

'                       '

In [329]:
df['Authors'].mode()

0       Alain Mabanckou
1        Barbara Demick
2             Bob Dylan
3            Chris Ware
4       Daniel Kahneman
5          Deborah Levy
6      Hanya Yanagihara
7         Helen Dunmore
8            JK Rowling
9       Jenny Erpenbeck
10            Jim Crace
11           Lorna Sage
12               M John
13     Malcolm Gladwell
14     Malorie Blackman
15           NK Jemisin
16          Neil Gaiman
17        Nicola Barker
18          Nora Ephron
19       Olga Tokarczuk
20    Patricia Lockwood
21      Richard Dawkins
22    Richard Wilkinson
23      Sebastian Barry
24      Shoshana Zuboff
25        Stieg Larsson
26           Ted Chiang
27     Valeria Luiselli
28     Yanis Varoufakis
29         Yuri Herrera
Name: Authors, dtype: object